# 패키지

In [1]:
import os                           # 운영체제
import tensorflow as tf             # 텐서플로
import pandas as pd                 # 판다스
import matplotlib.pyplot as plt     # 그래프 도구
import my_models as mymod           # 케라스 사용자 모델
import my_dataset as myds           # 데이터셋 생성기

/Users/chorokmoo/_MyDev/_Anaconda/anaconda3/envs/new2/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# GPU 설정

In [2]:
# 사용 가능한 모든 GPU 리스트
gpus = tf.config.list_physical_devices('GPU')

# 사용할 프로세서 선택 : GPU
if gpus:
    try: tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: print(e)

# 데이터셋

In [3]:
ds = myds.make()
ds_validation = ds.take(320).batch(32).prefetch(tf.data.experimental.AUTOTUNE)  # 훈련 데이터셋
ds_train = ds.skip(320).batch(32)                                               # 검증 데이터셋

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

생성된 데이터셋 정보
--------------------------------
shape 0   : (22617, 12)
--------------------------------
count 0   : [22502, 6044, 7567, 13256, 8180, 7618, 2011, 9, 14, 58, 15, 2]
--------------------------------
[12]


# 콜백 함수

In [4]:
# 모델 주소
model_dir = os.path.join('.', 'models', 'msrm.h5')

# 최상의 모델 저장
cb_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_dir, save_best_only=True)
cb_early_stop = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

# 손실 함수

In [5]:
class CustomLoss(tf.keras.losses.Loss):
    def __init__(self, numbers, **kwargs):
        numbers = [3.0 / n for n in numbers]
        total = sum(numbers)
        self.rate = [n / total for n in numbers]
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        # 자료형 통일
        y_true = tf.cast(y_true, dtype=y_pred.dtype)

        # 손실 계산
        #loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
        loss = tf.abs(y_true - y_pred)

        # 배치 크기 계산
        batch_size = tf.shape(y_pred)[0]

        # 계산된 가중치 브로드 개스팅
        weights = tf.broadcast_to(self.rate, [batch_size, tf.shape(self.rate)[0]])

        # 가중치를 사용하여 손실을 계산함
        weighted_losses = tf.reduce_sum(weights * loss, axis=-1)
        weighted_losses = tf.reduce_mean(weighted_losses)
        return weighted_losses

# 모델 생성 및 학습

In [ ]:
# 그래프 초기화
tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    # 모델 존재 확인
    if os.path.exists(model_dir): 
        # 모델 불러오기
        model = tf.keras.models.load_model(model_dir) 
    else:
        # ResNet50 모델 불러오기 (사전 학습된 가중치 사용)
        base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(512, 192, 3))

        # 새로운 분류층 추가
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        out = tf.keras.layers.Dense(12, activation='sigmoid')(x)

        # 모델 생성
        model = tf.keras.models.Model(inputs=base_model.input, outputs=out)

    # 사전 학습 층 동결
    for layer in base_model.layers:
        layer.trainable = False

    # 모델 학습 설정
    model.compile(
        optimizer='adam',
        loss=[CustomLoss([22502, 6044, 7567, 13256, 8180, 7618, 2011, 9, 14, 58, 15, 2])],
        metrics=['accuracy']
    )

    # 모델 훈련 및 저장
    history = model.fit(ds_train, epochs=10, initial_epoch=0, validation_data=ds_validation, callbacks=[cb_checkpoint, cb_early_stop])

    # 모델 저장
    model.save(model_dir)


    # 사전 학습 층 동결
    for layer in base_model.layers:
        layer.trainable = True

    # 모델 훈련 및 저장
    history = model.fit(ds_train, epochs=20, initial_epoch=10, validation_data=ds_validation, callbacks=[cb_checkpoint, cb_early_stop])

    # 모델 저장
    model.save(model_dir)

In [ ]:
# 그래프 초기화
tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    # 사전 학습 층 동결
    for layer in base_model.layers:
        layer.trainable = True

    # 모델 학습 설정
    model.compile(
        optimizer='adam',
        loss=['binary_crossentropy'],
        metrics=['accuracy']
    )

    # 모델 훈련 및 저장
    history = model.fit(ds_train, epochs=20, initial_epoch=10, validation_data=ds_validation, callbacks=[cb_checkpoint, cb_early_stop])

    # 모델 저장
    model.save(model_dir)

In [ ]:
# 그래프 초기화
tf.keras.backend.clear_session()

with tf.device('/device:GPU:0'):
    # 모델 존재 확인
    if os.path.exists(model_dir): model = tf.keras.models.load_model(model_dir)                         # 모델 불러오기
    else                        : model = mymod.msrm([12, 11, 14, 5, 9, 18, 8, 7, 12, 9, 14, 6, 15, 2]) # 모델 생성

    # 모델 학습 설정
    model.compile(
        optimizer='adam',
        loss=[
            'binary_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy',
            'categorical_crossentropy'
        ],
        metrics=['accuracy']
    )

    # 모델 훈련 및 저장
    history = model.fit(ds_train, epochs=10, initial_epoch=0, validation_data=ds_validation, callbacks=[cb_checkpoint, cb_early_stop])

    # 모델 저장
    model.save(model_dir)

# 학습 결과

In [ ]:
# 훈련 결과 그리기
pd.DataFrame(history.history).plot(figsize=(20, 10))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()